In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import spacy
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import numpy as np
import nltk
import random
from langdetect import detect
from pathlib import Path
from textblob import TextBlob
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def lemmatise_text(text):
    tweet_nlp = nlp(text)
    l = [token.lemma_ for token in tweet_nlp ]
    return (" ".join(l)).replace(" \n","")

def stem_text(text):
    words = word_tokenize(text)
    l = [stemmer.stem(token) for token in words]
    return " ".join(l)


In [3]:
nlp = spacy.load("C:/Users/Benco/Anaconda3/Lib/site-packages/fr_core_news_sm/fr_core_news_sm-2.2.5")
stemmer = SnowballStemmer('french')

In [6]:
train = pd.read_csv("train_data.csv")  #pas forcément à éxécuter
train = train[["question" , "reponse"]]
train["lemmas_rep"] = train["reponse"].apply(lemmatise_text)
train["lemmas_quest"] = train["question"].apply(lemmatise_text)
train["stem_rep"] = train["reponse"].apply(stem_text)
train["stem_quest"] = train["question"].apply(stem_text)

In [7]:
train.to_csv("training_data.csv" , index = False)

In [4]:
train = pd.read_csv("training_data.csv")
train

,question,reponse,lemmas_rep,lemmas_quest,stem_rep,stem_quest
0,Quels sont les forfaits vacances disponibles ?,Nos forfaits vacances répondent à tous vos bes...,notre forfait vacance répondre à tout votre be...,quel être le forfait vacance disponible ?,nos forfait vacanc répondent à tous vos besoin...,quel sont le forfait vacanc disponibl ?
1,Comment supprimer les cookies Flash ?,Voici comment modifier les paramètres Flash. S...,voici comment modifier le paramètre flash . so...,comment supprimer le cookie flash ?,voic comment modifi le parametr flash . souven...,comment supprim le cook flash ?
2,Quels types de restaurants trouve t-on dans le...,Disneyland Paris dispose de plus de 65 restaur...,Disneyland Paris dispose de plus de 65 restaur...,quel type de restaurant trouve t - on dans le ...,disneyland paris dispos de plus de 65 restaur ...,quel typ de restaur trouv t-on dan le parc dis...
3,Quels concerts y a-t-il actuellement au Disney...,Découvrez les superbes concerts qui se déroule...,Découvrez le superbe concert qui se dérouler b...,quel concerts y avoir t il actuellement au Dis...,découvr le superb concert qui se déroul bientô...,quel concert y a-t-il actuel au disney villag ?
4,Comment connaître la disponibilité des chambre...,Vous pouvez consulter la disponibilité des cha...,vous pouvoir consulter le disponibilité un cha...,comment connaître le disponibilité un chambre ...,vous pouv consult la disponibil de chambr d'hô...,comment connaîtr la disponibil de chambr avant...
...,...,...,...,...,...,...
139,Où se trouve le centre Accueil Animaux ?,Le centre Accueil Animaux se trouve à proximit...,le centre Accueil Animaux se trouve à proximit...,où se trouve le centre Accueil Animaux ?,le centr accueil animal se trouv à proxim du p...,où se trouv le centr accueil animal ?
140,"Je souffre d'intolérances alimentaires, où pui...",Pas d'inquiétude. Les restaurants Disney sont ...,pas de inquiétude . le restaurant Disney être ...,"je souffrir de intolérance alimentaire , où pu...",pas d'inquiétud . le restaur disney sont heure...,"je souffr d'intoler alimentair , où puis-j mang ?"
141,Est-il possible d’acheter un PhotoPass+ Annuel...,"Oui, selon les conditions tarifaires en vigueu...","oui , selon le condition tarifaire en vigueur ...",est - il possible d’ acheter un photopass+ ann...,"oui , selon le condit tarifair en vigueur au m...",est-il possibl d ’ achet un photopass+ annuel ...
142,Y a-t-il des poussettes dans les Parcs Disney ?,Oui. Il est possible de louer des poussettes a...,oui . il être possible de louer un poussette a...,y avoir t il un poussette dans le parc Disney ?,oui . il est possibl de lou de pousset au poin...,y a-t-il de pousset dan le parc disney ?


In [8]:
sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']

In [29]:
vectorizer = TfidfVectorizer(lowercase=True, stop_words = sw,
                            ngram_range=(1, 1),
                            min_df = 5,
                            use_idf=True, smooth_idf=True, # idf lissé
                            sublinear_tf=False, norm='l2')

In [30]:
X_train_vectorized_tfidf = vectorizer.fit_transform(pd.concat([train["lemmas_quest"].apply(stem_text) , train["lemmas_rep"].apply(stem_text)]))# compléter ici

In [31]:
dt = (pd.DataFrame(X_train_vectorized_tfidf.toarray(), columns=vectorizer.get_feature_names())).transpose()
dt.columns = ["doc_"+str(i) for i in range(288)]
dt

,doc_0,doc_1,doc_2,doc_3,doc_4,doc_5,doc_6,doc_7,doc_8,doc_9,...,doc_278,doc_279,doc_280,doc_281,doc_282,doc_283,doc_284,doc_285,doc_286,doc_287
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.00000,0.0,0.105859,0.0,0.0,0.000000,0.0
25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.10044,0.0,0.000000,0.0,0.0,0.000000,0.0
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0
300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0
âge,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.245088,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.192567,0.0
égal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.075746,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0
équip,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.14873,0.000000,0.000000,0.00000,0.0,0.109911,0.0,0.0,0.000000,0.0


In [38]:
query = [stem_text(lemmatise_text("Quel est le tarif pour les restaurants ?"))]
query_vector = vectorizer.transform(query)
pd.DataFrame(query_vector.toarray(), columns=vectorizer.get_feature_names())

,12,15,25,30,300,500,825,acced,acces,accompagn,...,votr,vouloir,véhicul,walt,web,zon,âge,égal,équip,éven
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
query_corpus_sim = np.squeeze(cosine_similarity(X_train_vectorized_tfidf, query_vector)) ## on enlève leur format qui n'est pas bon --> vecteur
query_corpus_sim
#np.max(query_corpus_sim)

array([0.23059406, 0.        , 0.49972111, 0.18722344, 0.        ,
       0.        , 0.        , 0.41832372, 0.        , 0.18762589,
       0.        , 0.        , 0.20823938, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.17106967, 0.35705375, 0.19345046, 0.        ,
       0.        , 0.        , 0.        , 0.16521989, 0.18871983,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.15876472, 0.17349555, 0.        , 0.16511592, 0.        ,
       0.        , 0.        , 0.13819596, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.18343577, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.23335314, 0.19397747,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.2023209 , 0.        , 0.        , 0.        , 0.     

In [36]:
id_best_doc = np.argsort(query_corpus_sim)[-1]
id_best_doc

2

In [37]:
 ## ordre croissant
if(id_best_doc >= 144):
    doc = train["reponse"][id_best_doc-144]
else:
    doc = train["reponse"][id_best_doc]
doc 

"Disneyland Paris dispose de plus de 65 restaurants et bars aux spécialités et tarifs divers. Vous pouvez découvrir tout un monde de délices culinaires, des snacks aux stands jusqu'aux dîners gourmets. Pour les gourmands, des buffets Tex-Mex et des barbecues vous sont proposés dans des cadres à thème. Vous pouvez également vous restaurer dans des chaînes internationales comme McDonalds et Starbucks. Vous pouvez même partager un repas avec des personnages Disney : réservez-le dès aujourd'hui !En savoir plus sur nos menus enfants et nos solutions pour les personnes ayant des besoins alimentaires particuliers"